## Setup Steps

# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

In [0]:
#@title Please enter your registration id to start: (e.g. P181900101) { run: "auto", display-mode: "form" }
Id = "P181902118" #@param {type:"string"}


In [0]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "8860303743" #@param {type:"string"}


In [0]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()
  
notebook="M3W9_CS_3_Author_identification_RNN" #name of the notebook
Answer = "Ungraded"
def setup():
#  ipython.magic("sx pip3 install torch") 
    ipython.magic("sx pip3 install gensim")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/week12/Exp2/test.csv")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/week12/Exp2/train.csv")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/week12/Exp2/val.csv")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/CaseStudies/Sentence_level_rnn_trained_0.66.pt")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook}

      r = requests.post(url, data = data)
      r = json.loads(r.text)
      print("Your submission is successful.")
      print("Ref Id:", submission_id)
      print("Date of submission: ", r["date"])
      print("Time of submission: ", r["time"])
      print("View your submissions: https://iiith-aiml.talentsprint.com/notebook_submissions")
      print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
      return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if Additional: return Additional      
    else: raise NameError('')
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getAnswer():
  try:
    return Answer
  except NameError:
    print ("Please answer Question")
    return None

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
    from IPython.display import HTML
    HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id))
  
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


In [0]:
ls

sample_data/


###Importing required packages

In [0]:
import pandas as pd
import numpy as np
import gensim
import re

In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

### Assign numbers to the labels using dictionary data structure


In [0]:
label = {'EAP':0, 'HPL':1, 'MWS':2}

### Load the train data

In [0]:
train_data = pd.read_csv('train.csv', encoding='latin1')

In [0]:
train_data.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


### Collect the sentences from the train data

In [0]:
sentences = train_data[['text','author']].values

### Load the test data

In [0]:
test_data = pd.read_csv('test.csv', encoding='latin1')

### Collect the sentences from the test data

In [0]:
test_sentences = test_data[['text']].values.flatten()

### Define the class for stemming / preprocessing sentences

In [0]:
#stopWords = pd.read_csv('stopwords.txt').values

class MySentences(object):
    def __init__(self, fnamelist):
        self.fnamelist = fnamelist
        # Creating a set of vocabulary
        self.vocabulary = set([])
        #self.sentences = self.train_data.text.values
        #self.labels = [self.train_data.text.values.flatten()

    def __iter__(self):
        for fname in self.fnamelist:
            self.data = pd.read_csv(fname, encoding='latin1')
            self.sentences = self.data.text.values
            for line in self.sentences:
                words = re.findall(r'(\b[A-Za-z][a-z]{2,15}\b)', line)
                # Stemming a word.
                words = [ word.lower() for word in words]
                for word in words:
                    self.vocabulary.add(word)
                yield words

In [0]:
sentences = MySentences(['train.csv', 'val.csv','test.csv']) # a memory-friendly iterator
# for i in sentences:
#     print(i)

### Use gensims.model.Word2Vec to get vectors for the sentences and save the model as a .bin file

In [0]:
model = gensim.models.Word2Vec(sentences, min_count=1)
model.save("AuthID2Vec.bin")

### Count the corpus

In [0]:
model.corpus_count

27971

### Convert the sentences to lower case

In [0]:
for i, _d in enumerate(test_sentences):
    try:
        _d.lower()
    except:
        print(_d)

### Import required torch packages

In [0]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import os

### Define custom dataset loader

In [0]:
class CustomDataset(torch.utils.data.Dataset):    ### custom data loader
    
    def __init__(self, data_file_path,  train=True):
        self.data_file_path = data_file_path
        self.train = train
        self.data = pd.read_csv(data_file_path, encoding='latin1')
        self.ids = self.data.id.values
        self.sentences = self.data.text.values
        if self.train:
            self.label_dict = {'EAP':0, 'HPL':1, 'MWS':2}
            self.labels = [self.label_dict[i] for i in self.data.author.values]
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence = self.sentences[index]
        if self.train:
            return self.sentences[index], self.labels[index]
        else:
            return self.sentences[index], self.ids[index]

### Set the batch size

In [0]:
batch_size = 16

### Use the custom dataset loader to load the .csv files for train, val and test data into batches

In [0]:
# You can then use the prebuilt data loader. 
train_set = CustomDataset("train.csv", train=True)
val_set = CustomDataset("val.csv", train=True)
test_set = CustomDataset("test.csv", train=False)

In [0]:
trainloader = torch.utils.data.DataLoader(train_set,batch_size=batch_size, shuffle = True)
valloader = torch.utils.data.DataLoader(val_set,batch_size=batch_size, shuffle = True)
testloader = torch.utils.data.DataLoader(test_set,batch_size=batch_size, shuffle = False)

In [0]:
for X,y in trainloader:
    print(X)
    print(y.size())
    break

('A full thousand cubits high stood the greatest among them, wherein the high priests dwelt with a magnificence scarce less than that of the kings.', 'The left arm of the figure is bent at the elbow, and in the left hand is a pipe.', 'He plucks flowers and weeds, and weaves chaplets of them, or sails yellow leaves and bits of bark on the stream, rejoicing in their safety, or weeping at their wreck.', "To me Alos denied a warrior's part, for I was feeble and given to strange faintings when subjected to stress and hardships.", 'Turning to the number which designated the oval portrait, I there read the vague and quaint words which follow: "She was a maiden of rarest beauty, and not more lovely than full of glee.', 'Here the bank slopes upward from the stream in a very gentle ascent, forming a broad sward of grass of a texture resembling nothing so much as velvet, and of a brilliancy of green which would bear comparison with the tint of the purest emerald.', 'Chapter I sat one evening in m

### Load the saved gensim model which contains vectors for the sentences 

In [0]:
model_load = gensim.models.Word2Vec.load('AuthID2Vec.bin')

### Set the deivce to CUDA

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Creating the recurrent neural network

In [0]:
### Creating recurrent neural network
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.gru = nn.GRU(input_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax()
    
    def forward(self, input):
        outputs = []
        for sentence in input:
            hidden= self.init_hidden()
            word_embeddings = self.get_embedding(sentence)
            for word_embedding in word_embeddings:
                output, hidden = self.gru(word_embedding.unsqueeze(0).unsqueeze(0), hidden)
            try:
                #print(output.size())
                outputs.append(output)
            except:
                outputs.append(torch.rand(1,1,20))
                continue
        outputs = torch.cat(outputs)
        outputs = self.softmax(self.decoder(outputs))
        return outputs

    def init_hidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size)
    
    def get_embedding(self, sentence):
        #print(len(sentence))
        #sentence_wt = word_tokenize(sentence.lower())
        words = re.findall(r'(\b[A-Za-z][a-z]{2,15}\b)', sentence)
        words = [ word.lower() for word in words]
        embedding = []
        for word in words:
            #print(word)
            embedding.append(torch.tensor(model_load[word]))
        #print(embedding[0].size())
        return embedding

### Implement the RNN by setting up the required parameters

In [0]:
rnn = RNN(100,20,3,n_layers=1) #Set the denfined RNN model with 100 input layers, 20 hidden layers and 3 output layers
lr = 0.001 # learning rate
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr) # Set the optimizer 
## Loss function
criterion = nn.CrossEntropyLoss()

### Train and test RNN model

### This will take quite a lot of time

In [0]:
losses  = []
val_accuracy = 0
for j in range(100):
    i = 0
    correct_train = 0
    for X,y in trainloader:
        #print(index, end='\r')
        output = rnn(X)
        optimizer.zero_grad()
        loss = criterion(output.squeeze(1),y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        _,predicted = torch.max(output.squeeze(1).data, 1)
        #print(predicted)
        correct_train += predicted.eq(y.data).cpu().sum().item()
        i=i+1
    print('Epoch: {}, Train Accuracy: {}, Average Loss: {}'.format(j, correct_train/(i*16), sum(losses)/len(losses)))
    correct_val = 0 
    for X,y in valloader:
        #print(index, end='\r')
        output = rnn(X)
        _,predicted = torch.max(output.squeeze(1).data, 1)
        correct_val += predicted.eq(y.data).cpu().sum().item()
    if val_accuracy<correct_val/6580:
        val_accuracy = correct_val/6580
        torch.save(rnn.state_dict(), 'Sentence_level_rnn_trained_{:.2f}.pt'.format(val_accuracy))
    print('Epoch: {}, Validation Accuracy: {}'.format(j, correct_val/6580))

Epoch: 0, Train Accuracy: 0.4690959409594096, Average Loss: 1.0754981806357409
Epoch: 0, Validation Accuracy: 0.5048632218844985
Epoch: 1, Train Accuracy: 0.5306734317343174, Average Loss: 1.0657202297910993
Epoch: 1, Validation Accuracy: 0.5451367781155015
Epoch: 2, Train Accuracy: 0.548739237392374, Average Loss: 1.059102670366327
Epoch: 2, Validation Accuracy: 0.5534954407294833
Epoch: 3, Train Accuracy: 0.5589637146371463, Average Loss: 1.054734205792751
Epoch: 3, Validation Accuracy: 0.5547112462006079
Epoch: 4, Train Accuracy: 0.5672662976629766, Average Loss: 1.0515190838887682
Epoch: 4, Validation Accuracy: 0.5566869300911854
Epoch: 5, Train Accuracy: 0.5711869618696187, Average Loss: 1.049112016619987
Epoch: 5, Validation Accuracy: 0.5688449848024316
Epoch: 6, Train Accuracy: 0.5828720787207872, Average Loss: 1.0469419210494622
Epoch: 6, Validation Accuracy: 0.579483282674772
Epoch: 7, Train Accuracy: 0.5824876998769988, Average Loss: 1.04515111553603
Epoch: 7, Validation Accu

### Please answer the questions below to complete the experiment:

In [0]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good and Challenging for me" #@param ["Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [0]:
#@title If it was very easy, what more you would have liked to have been added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "test'\"" #@param {type:"string"}

In [0]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["Yes", "No"]

In [0]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id =return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")

Your submission is successful.
Ref Id: 5320
Date of submission:  24 May 2019
Time of submission:  10:32:50
View your submissions: https://iiith-aiml.talentsprint.com/notebook_submissions
For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.
